Agrupacion y verificacion del dataset

In [ ]:
import os
import sys
from tqdm import tqdm

BASE_INPUT_DIR = None
original_biome_names = []

try:
    dataset_root_walk = next(os.walk('/kaggle/input')) 
    if not dataset_root_walk[1] and dataset_root_walk[0] == '/kaggle/input':
         dataset_root_walk = next(os.walk(dataset_root_walk[0]))

    dataset_root_dir = dataset_root_walk[0]
    preprocessed_data_path = os.path.join(dataset_root_dir, 'preprocessed_data')
    
    if os.path.isdir(preprocessed_data_path):
        BASE_INPUT_DIR = preprocessed_data_path
        original_biome_names = next(os.walk(BASE_INPUT_DIR))[1]
        print(f"Directorio base de entrada detectado: {BASE_INPUT_DIR}")
        print(f"Se encontraron {len(original_biome_names)} directorios 'biome_X'.")
    else:
        print("Advertencia: No se encontró 'preprocessed_data'. Buscando biomas en el directorio raíz del dataset.")
        BASE_INPUT_DIR = dataset_root_dir
        original_biome_names = dataset_root_walk[1]
        if not any(name.startswith('biome_') for name in original_biome_names):
            raise FileNotFoundError("No se encontró 'preprocessed_data' ni directorios 'biome_X' en la raíz.")
            
except (StopIteration, FileNotFoundError) as e:
    print(f"Error: No se encontró una estructura de directorios válida. {e}")
    print("Asegúrate de que el dataset (con 'preprocessed_data') esté agregado a este notebook.")

BASE_OUTPUT_DIR = '/kaggle/working/dataset_agrupado'
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
print(f"El dataset agrupado se guardará en: {BASE_OUTPUT_DIR}")


BIOME_MAP_BY_ID = {
    "Taiga": ["biome_5", "biome_19", "biome_32", "biome_33", "biome_133"],
    "Taiga Nevada": ["biome_30", "biome_31", "biome_158"],
    "Savana": ["biome_35", "biome_36"],
    "Jungla": ["biome_21", "biome_22"],
    "Bosque de Roble Oscuro": ["biome_29", "biome_157"],
    "Desierto": ["biome_2", "biome_17", "biome_130"],
    "Badlands": ["biome_37", "biome_38", "biome_39"],
    "Bosque de Abeto": ["biome_27", "biome_28", "biome_156"],
    "Pantano": ["biome_6"],
    "Bosque de Roble": ["biome_4", "biome_132"],
    "Planicies": ["biome_1", "biome_129"],
    "Bosque Mixto": ["biome_18", "biome_34"],
    "Tundra Nevada": ["biome_12"],
    "Montañas": ["biome_3", "biome_131", "biome_162"],
    "Montaña Nevada": ["biome_13"],
    "Playa": ["biome_16", "biome_26"],
    "Ríos": ["biome_7", "biome_11"]
}

BIOMAS_EXCLUIDOS_BY_ID = ["biome_10", "biome_45"]

total_images_processed = 0
new_category_counts = {category: 0 for category in BIOME_MAP_BY_ID.keys()}
original_biomes_mapped = set()

for new_category, original_biome_id_list in tqdm(BIOME_MAP_BY_ID.items(), desc="Agrupando categorías"): 
    new_category_dir = os.path.join(BASE_OUTPUT_DIR, new_category)
    os.makedirs(new_category_dir, exist_ok=True)
    
    for old_biome_dir_name in original_biome_id_list:
        original_biome_path = os.path.join(BASE_INPUT_DIR, old_biome_dir_name)
        original_biomes_mapped.add(old_biome_dir_name)
        
        if not os.path.isdir(original_biome_path):
            print(f"  -> Advertencia: No se encontró el directorio para '{old_biome_dir_name}'. Omitiendo.")
            continue
            
        try:
            images = os.listdir(original_biome_path)
            for image_name in images:
                source_path = os.path.join(original_biome_path, image_name)
                dest_name = f"{old_biome_dir_name}_{image_name}"
                dest_path = os.path.join(new_category_dir, dest_name)
                
                try:
                    os.symlink(source_path, dest_path)
                    total_images_processed += 1
                    new_category_counts[new_category] += 1
                except FileExistsError:
                    pass 
                except Exception as e:
                    print(f"Error al crear enlace para {source_path}: {e}")

        except Exception as e:
            print(f"  -> Error procesando '{original_biome_path}': {e}")

print("\n--- Proceso de agrupación completado ---")
print(f"Total de imágenes enlazadas: {total_images_processed}")

target_counts = {
    "Taiga": 2346, "Taiga Nevada": 420, "Savana": 1164, "Jungla": 444,
    "Bosque de Roble Oscuro": 870, "Desierto": 2088, "Badlands": 210,
    "Bosque de Abeto": 1122, "Pantano": 665, "Bosque de Roble": 2850,
    "Planicies": 3186, "Bosque Mixto": 1320, "Tundra Nevada": 1254,
    "Montañas": 2652, "Montaña Nevada": 474, "Playa": 540, "Ríos": 444
}
print("\n--- Verificación de Conteo de Imágenes ---")
for category, count in new_category_counts.items():
    target = target_counts.get(category, 0)
    discrepancy = ""
    if count != target:
        discrepancy = f" (Esperado: {target} - ¡Revisar!)"
    print(f"- {category}: {count} imágenes {discrepancy}")

all_original_biomes_in_map = original_biomes_mapped | set(BIOMAS_EXCLUIDOS_BY_ID)
unmapped_biomes = [b for b in original_biome_names if b not in all_original_biomes_in_map]
        
if unmapped_biomes:
    print(f"\n¡ADVERTENCIA! Los siguientes directorios no fueron mapeados ni excluidos:")
    for b in unmapped_biomes: print(f"  - {b}")
else:
    print("\nTodos los directorios de biomas originales fueron mapeados o excluidos correctamente.")

Division de los datos para entrenamiento y prueba

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics
import seaborn as sns
import os
import sys

IMG_HEIGHT = 180
IMG_WIDTH = 320
BATCH_SIZE = 32
VALIDATION_SPLIT = 1/3 
SEED = 123
DATA_DIR = '/kaggle/working/dataset_agrupado'

if not os.path.exists(DATA_DIR) or not os.listdir(DATA_DIR):
    print(f"Error: El directorio '{DATA_DIR}' está vacío o no existe.")
    print("Por favor, ejecuta la Celda 1 (agrupación de dataset) primero.")
else:
    CLASS_NAMES = sorted(os.listdir(DATA_DIR))
    NUM_CLASSES = len(CLASS_NAMES)
    print(f"Configuración lista.")
    print(f"División de datos: {1-VALIDATION_SPLIT:.2f} Entreno / {VALIDATION_SPLIT:.2f} Prueba")
    print(f"Clases detectadas ({NUM_CLASSES}): {CLASS_NAMES}")

Normalisacion del dataset haciendo los cambios

In [ ]:
target_counts = {
    "Taiga": 2346, "Taiga Nevada": 420, "Savana": 1164, "Jungla": 444,
    "Bosque de Roble Oscuro": 870, "Desierto": 2088, "Badlands": 210,
    "Bosque de Abeto": 1122, "Pantano": 665, "Bosque de Roble": 2850,
    "Planicies": 3186, "Bosque Mixto": 1320, "Tundra Nevada": 1254,
    "Montañas": 2652, "Montaña Nevada": 474, "Playa": 540, "Ríos": 444
}

MINORITY_THRESHOLD = 1000
print(f"Umbral para aumento de datos: {MINORITY_THRESHOLD} imágenes.")

print("Cargando dataset de entrenamiento (2/3)...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

print("Cargando dataset de prueba (1/3)...")
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names_loaded = train_ds.class_names
minority_class_indices = []

print("Identificando clases minoritarias para aumento:")
for i, class_name in enumerate(class_names_loaded):
    count = target_counts.get(class_name, 0)
    if count < MINORITY_THRESHOLD and count > 0:
        minority_class_indices.append(i)
        print(f"  -> SÍ: {class_name} (índice {i}) con {count} imágenes.")
    else:
        print(f"  -> NO: {class_name} (índice {i}) con {count} imágenes.")

MINORITY_INDICES_TENSOR = tf.constant(minority_class_indices, dtype=tf.int32)

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.1),
    ],
    name="data_augmentation",
)

@tf.function
def augment_if_minority(x, y):
    x_augmented = data_augmentation(x, training=True)
    y_cast = tf.cast(y, tf.int32)
    y_reshaped = tf.reshape(y_cast, [-1, 1])
    minority_reshaped = tf.reshape(MINORITY_INDICES_TENSOR, [1, -1])   
    comparison_matrix = tf.equal(y_reshaped, minority_reshaped)
    is_minority_vec = tf.reduce_any(comparison_matrix, axis=1)  
    condition = tf.reshape(is_minority_vec, [-1, 1, 1, 1])
    x_final = tf.where(condition, x_augmented, x)
    return x_final, y

print("\nAplicando aumento condicional SOLO a las clases minoritarias...")

train_ds = train_ds.map(
    augment_if_minority,
    num_parallel_calls=tf.data.AUTOTUNE
)

train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("\nDatasets de entrenamiento (condicional) y prueba listos.")

Constructor del modelo CNN

In [ ]:
def build_model(num_classes, architecture_type="estandar"):
    model = models.Sequential()
    model.add(layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(layers.Rescaling(1./255)) # Normalización

    if architecture_type == "estandar":
        model.name = "BiomeClassifier_Estandar"
        model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(512, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dropout(0.3))

    elif architecture_type == "simple":
        model.name = "BiomeClassifier_Simple"
        model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))   
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dropout(0.5))
        
    else:
        raise ValueError("Tipo de arquitectura no reconocido. Use 'estandar' o 'simple'.")

    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

print("Función 'build_model' (flexible) definida.")

Entrenamiento del modelo 

In [ ]:
def train_model_interactive(model, train_ds, val_ds, existing_history=None):
    try:
        epochs = int(input("Ingrese el número de épocas (ej. 10): ") or 10)
        lr = float(input("Ingrese la tasa de aprendizaje (ej. 0.001): ") or 0.001)
    except ValueError:
        print("Error: Entrada no válida.")
        return model, existing_history

    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"\nEntrenando por {epochs} épocas con LR={lr}...")
    
    history = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=val_ds,
        verbose=1
    )
    
    print("Entrenamiento finalizado.")
    
    if existing_history:
        for key in history.history:
            if key in existing_history.history:
                existing_history.history[key].extend(history.history[key])
            else:
                existing_history.history[key] = history.history[key]
        plot_loss_history(existing_history)
        return model, existing_history
    else:
        plot_loss_history(history)
        return model, history

def plot_loss_history(history):
    acc = history.history.get('accuracy', [])
    val_acc = history.history.get('val_accuracy', [])
    loss = history.history.get('loss', [])
    val_loss = history.history.get('val_loss', [])
    
    if not acc or not val_acc or not loss or not val_loss:
        print("No hay suficiente historial para graficar.")
        return

    epochs_range = range(len(acc))
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Precisión de Entrenamiento')
    plt.plot(epochs_range, val_acc, label='Precisión de Prueba')
    plt.legend(loc='lower right')
    plt.title('Precisión de Entrenamiento y Prueba')
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Pérdida de Entrenamiento')
    plt.plot(epochs_range, val_loss, label='Pérdida de Prueba')
    plt.legend(loc='upper right')
    plt.title('Pérdida de Entrenamiento y Prueba')
    plt.show()

def test_model_interactive(model, val_ds, class_names_list):
    print("Evaluando modelo contra el conjunto de prueba (1/3)...")
    loss, accuracy = model.evaluate(val_ds, verbose=0)
    print("\n" + "="*60)
    print("          RESULTADOS DE LA EVALUACIÓN")
    print("="*60)
    print(f"Pérdida (Loss): {loss:.4f}")
    print(f"Precisión (Accuracy): {accuracy * 100:.2f}%")
    if accuracy > 0.90:
        print("¡Éxito! Se alcanzó el objetivo de >90% de precisión.")
    else:
        print(f"Objetivo de >90% no alcanzado.")
    print("="*60)

    print("\nGenerando predicciones para el reporte detallado...")
    y_pred = []
    y_true = []
    for images, labels in val_ds:
        y_true.extend(labels.numpy())
        preds = model.predict(images, verbose=0)
        y_pred.extend(np.argmax(preds, axis=1))
      
    print("\n--- Reporte de Clasificación (Precisión, Recall, F1) ---")
    print(metrics.classification_report(y_true, y_pred, target_names=class_names_list, zero_division=0))
    print("Generando Matriz de Confusión...")
    cm = metrics.confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(14, 12))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues', 
        xticklabels=class_names_list, 
        yticklabels=class_names_list
    )
    plt.title('Matriz de Confusión')
    plt.ylabel('Bioma Real (True Label)')
    plt.xlabel('Bioma Predicho (Predicted Label)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.show()

def save_model_interactive(model):
    print("El modelo se guardará en '/kaggle/working/'")
    filename = input("Nombre del archivo (ej: mi_modelo.keras): ")
    
    if not filename:
        print("Guardado cancelado.")
        return
        
    save_path = os.path.join("/kaggle/working/", filename)
    
    try:
        model.save(save_path)
        print(f"Modelo guardado exitosamente en {save_path}")
    except Exception as e:
        print(f"Error al guardar el modelo: {e}")

def load_model_interactive():
    print("Buscando modelos en '/kaggle/working/' y '/kaggle/input/...")
    filename = input("Ruta completa del archivo a cargar (ej: /kaggle/working/mi_modelo.keras): ")

    if not filename:
        print("Carga cancelada.")
        return None

    if not os.path.exists(filename):
        print(f"Error: El archivo '{filename}' no existe.")
        return None
        
    try:
        model = models.load_model(filename)
        print(f"Modelo cargado exitosamente desde {filename}")
        model.summary()
        return model
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
        return None

def predict_single_image_interactive(model, class_names_list):
    print("\n--- Predecir Imagen Individual ---")
    print("Instrucciones: Use la barra lateral de Kaggle (Data > Upload) para subir su imagen.")
    print("La imagen aparecerá en '/kaggle/working/'.")
    path = input("Ingrese la ruta de la imagen (ej: /kaggle/working/mi_foto.png): ")

    if not os.path.exists(path):
        print(f"Error: No se encontró el archivo en '{path}'.")
        return

    try:
        img = tf.keras.utils.load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = tf.keras.utils.img_to_array(img)
        img_batch = tf.expand_dims(img_array, 0) # Crear un lote de 1
        predictions = model.predict(img_batch, verbose=0)
        score = tf.nn.softmax(predictions[0])  
        predicted_index = np.argmax(score)
        predicted_class = class_names_list[predicted_index]
        confidence = 100 * np.max(score)

        print("\n--- Resultado ---")
        print(f"La red cree que es: {predicted_class}")
        print(f"Confianza: {confidence:.2f}%")

    except Exception as e:
        print(f"Error al procesar la imagen: {e}")

print("Funciones interactivas (con predicción individual) definidas.")

Menu principal

In [ ]:
model = None
history = None

if 'train_ds' not in locals() or 'val_ds' not in locals() or 'CLASS_NAMES' not in locals():
    print("Error: Los datasets (train_ds, val_ds) no están cargados.")
    print("Por favor, asegúrate de ejecutar las Celdas 2 y 3 primero.")
else:
    print("Datasets (train_ds, val_ds) y función 'build_model' listos.")
    arch_choice = input("Creación automática: ¿Qué arquitectura desea? (1: Estándar, 2: Simple) [Default: 1]: ") or "1"
    arch_type = "simple" if arch_choice == "2" else "estandar"

    print(f"\nCreando arquitectura de modelo inicial ({arch_type})...")
    model = build_model(NUM_CLASSES, architecture_type=arch_type)
    history = None
    print("¡Nuevo modelo creado y listo para entrenar!")
    model.summary()
    
    while True:
        print("\n-------------------------------------------")
        print("  Panel de Control - Clasificador de Biomas")
        print("-------------------------------------------")
        
        if model is None:
            print("Estado: No hay modelo cargado en memoria.")
        else:
            print(f"Estado: Modelo '{model.name}' cargado.")
        print("\n--- Opciones Principales ---")
        print("1. Crear un nuevo modelo (borra el actual)")
        print("2. Cargar un modelo desde archivo")
        print("3. Entrenar el modelo actual")
        print("4. Evaluar (Probar) el modelo actual")
        print("5. Guardar el modelo actual")
        print("6. Predecir imagen individual")
        print("7. Salir")
        
        main_choice = input("Seleccione una opción: ")
        
        if main_choice == '1':
            arch_choice_menu = input("¿Qué arquitectura desea? (1: Estándar, 2: Simple) [Default: 1]: ") or "1"
            arch_type_menu = "simple" if arch_choice_menu == "2" else "estandar"
            print(f"\nCreando nueva arquitectura de modelo ({arch_type_menu})...")
            model = build_model(NUM_CLASSES, architecture_type=arch_type_menu)
            history = None
            print("¡Nuevo modelo creado y listo para entrenar!")
            model.summary()

        elif main_choice == '2':
            loaded_model = load_model_interactive()
            if loaded_model is not None:
                model = loaded_model
                history = None
                print("Modelo reemplazado por el archivo cargado.")

        elif main_choice == '3':
            if model is None:
                print("Error: No hay modelo en memoria. Cree (Opción 1) o cargue (Opción 2) un modelo.")
            else:
                model, history = train_model_interactive(model, train_ds, val_ds, history)

        elif main_choice == '4':
            if model is None:
                print("Error: No hay modelo en memoria. Cree (Opción 1) o cargue (Opción 2) un modelo.")
            else:
                test_model_interactive(model, val_ds, CLASS_NAMES)            
        elif main_choice == '5':
            if model is None:
                print("Error: No hay modelo en memoria para guardar.")
            else:
                save_model_interactive(model)
        elif main_choice == '6':
            if model is None:
                print("Error: No hay modelo en memoria. Cree (Opción 1) o cargue (Opción 2) un modelo.")
            else:
                predict_single_image_interactive(model, CLASS_NAMES)
        elif main_choice == '7':
            print("Saliendo del panel de control...")
            break
            
        else:
            print("Opción no válida. Por favor, intente de nuevo.")

print("Bucle del menú finalizado.")